In [6]:
#READ DATA
using CSV, DataFrames, JuMP, Gurobi

df_nut = CSV.read("data_nutrients.csv", DataFrame, delim = ";")
df_rec = CSV.read("data_recipes.csv", DataFrame, delim = ";")
df_price = CSV.read("price_ing.csv", DataFrame, delim = ";")
df_recConst = CSV.read("recipes_constr_cleaned_abbr.csv", DataFrame)

,Name,Servings,OrigIngredient
,String,Int64,String
1,Avocado and Corn Salsa,5,1 avocado (diced)
2,Avocado and Corn Salsa,5,"3/4 cup Frozen corn kernels, thawed"
3,Avocado and Corn Salsa,5,"1/2 cup grape tomatoes, quartered"
4,Avocado and Corn Salsa,5,"1 tablespoon fresh cilantro, chopped"
5,Avocado and Corn Salsa,5,2 teaspoons lime juice
6,Avocado and Corn Salsa,5,1/4 teaspoon salt
7,Baked Pork Chops,6,"6 pork chops (lean center-cut, 1/2-inch thick)"
8,Baked Pork Chops,6,"1 onion (medium, thinly sliced)"
9,Baked Pork Chops,6,1/2 cup green pepper (chopped)


In [83]:
# Create dictionary that maps recipe names to lists of ingredients
mydict = Dict(
    Pair.(
        transform(combine(groupby(df_recConst, :Name), :Match => Set)).Name,
        transform(combine(groupby(df_recConst, :Name), :Match => Set)).Match_Set
    )
)

Dict{String, Set{String}} with 218 entries:
  "Soy Marinated Chicken"   => Set(["chicken breast", "blood oranges", "kikkoma…
  "Caribbean Bean Salad"    => Set(["red onions", "tomato medley", "badia groun…
  "Rice Pudding"            => Set(["purified water", "equal exchange french va…
  "Roasted Herb Potatoes"   => Set(["badia garlic salt", "red potatoes", "badia…
  "Vegetable Snake"         => Set(["grape tomatoes", "cucumber"])
  "Lite Fried Okra"         => Set(["regal nonpareil cut okra  frozen"])
  "Spring Vegetable Soup"   => Set(["purified water", "imagine foods low sodium…
  "New England Johnny Cake" => Set(["best yet baking powder", "best yet granula…
  "Seared Greens"           => Set(["purified water", "badia whole black pepper…
  "Butternut Squash with B… => Set(["purified water", "goya black beans", "badi…
  "Granola Bars"            => Set(["best yet quick oats", "best yet honey bear…
  "Avocado and Corn Salsa"  => Set(["grape tomatoes", "limes", "best yet frozen…
  "Fru

In [109]:
# Create dictionary that maps meal number (1 = breakfast, 2 = lunch, 3 = dinner) to possible recipes
stacked = subset(
    stack(
        unique(select(df_recConst, :Name, :B, :L, :D, :S)), 
        2:5
    ),
    :value => ByRow(value -> value),
    skipmissing=true
)

temp_dict = Dict(
    Pair.(
        transform(combine(groupby(stacked, :variable), :Name => Set)).variable,
        transform(combine(groupby(stacked, :variable), :Name => Set)).Name_Set
    )
)

mydict2 = Dict(
    1 => temp_dict["B"],
    2 => temp_dict["L"],
    3 => temp_dict["D"],
    # 4 = > temp_dict["S"] # Uncomment to include snack/dessert items
)

Dict{Int64, Set{String}} with 3 entries:
  2 => Set(["Soy Marinated Chicken", "Caribbean Bean Salad", "Roasted Herb Pota…
  3 => Set(["Soy Marinated Chicken", "Caribbean Bean Salad", "Roasted Herb Pota…
  1 => Set(["Cinnamon Raisin Almond Balls", "Pear Rabbit", "Fruit Salad with Yo…

In [5]:
unique(df_recConst[:,1])

218-element Vector{String}:
 "3-Can Chili"
 "5 A Day Salad"
 "A Simple Mexican Salad"
 "Apple Carrot Soup"
 "Apple Chunk Cake"
 "Apple Oatmeal Muffins"
 "Apple Salad with Sweet & Tangy Yogurt Dressing"
 "Apple Sandwiches"
 "Apple Slice Pancakes"
 "Apple-Stuffed Squash"
 ⋮
 "Whole Grain Strawberry Pancakes"
 "Whole Wheat Garlic Bread Sticks"
 "Whole Wheat Muffins"
 "Yogurt Berry Parfait"
 "Yummy Bean Dip"
 "Zesty Citrus Coleslaw"
 "Zesty Orange Carrots"
 "Zucchini And Tomatoes"
 "Zucchini Bread"

In [80]:
#DEFINE SETS AND INDICES

#----------------------------------------------------------------------
#set of days (d)
d = 2
D = [j for j in range(1,d)]

#set of meals (m)
m = 3
M = [j for j in range(1,m)]

#set of nutrients (n)
N = unique(df_nut, "nutrients").nutrients

#set of nutrients with daily upper bound
uppnut = ["carb","protein","fat"]

#set of nutrients with per meal lower and upper bound
uppnutmeal = ["calories"]

#set of recipes (r)
# R = df_rec[:,1]
R = unique(df_recConst[:,1]) # Switch to use the recipe constraints to define available recipes

#set of people (p)
p = 2
P = [j for j in range(1,p)]

#set of ingredients (i)
I = unique(df_recConst, "need").need

# #subset of ingredients included in recipe r
# mydict = Dict("chicken_with_potatoes" => ["chicken","potatoes"], "beef_with_rice" => ["beef","rice"], "oatmeal" => ["oats"], "pancakes" => ["milk"])

# #subset of recipes for meal m
# mydict2 = Dict(1 => ["oatmeal","pancakes"], 2 => ["chicken_with_potatoes","beef_with_rice"], 3 => ["chicken_with_potatoes","beef_with_rice"])

#----------------------------------------------------------------------
#indice rmd
rmd = [(l,j,k) for l in R for j in M for k in D if l in mydict2[j]]

#indice rmdp
rmdp = [(l,j,k,f) for l in R for j in M for k in D for f in P if l in mydict2[j]]

#indice np
np = [(u,f) for u in N for f in P]

#indice nr
nr =[(u,l) for u in N for l in R]

#indice ir
ir = [(row.need,row.recipe) for row in eachrow(df_recConst)]

6-element Vector{Tuple{String15, String31}}:
 ("chicken", "chicken_with_potatoes")
 ("potatoes", "chicken_with_potatoes")
 ("beef", "beef_with_rice")
 ("rice", "beef_with_rice")
 ("oats", "oatmeal")
 ("milk", "pancakes")

In [99]:
#DEFINE DATA

#daily needs of nutrient n by person p
need = Dict((j,k) => df_nut[(df_nut.nutrients .== j) .& (df_nut.person .== k), :lowerbound][1] for (j,k) in np)

#daily max of nutrient n by person p
#max = Dict((j,k) => df_nut[(df_nut.nutrients .== j) .& (df_nut.person .== k), :upperbound][1] for (j,k) in np)
max = Dict((j,k) => df_nut[(df_nut.nutrients .== j) .& (df_nut.person .== k), :upperbound][1] for j in uppnut for k in P)

#price of ingredient i per unit of measurement of the ingredient
price = Dict(j => df_price[(df_price.ingredient .== j), :price][1] for j in I)

#quantity of nutrient n granted by one portion of recipe r
nutrec = Dict((j,k) => df_rec[(df_rec.recipe .== k), j][1] for (j,k) in nr)

#quantity of ingredient i needed by one portion of recipe r
recing = Dict((j,k) => df_recConst[(df_recConst.recipe .== k) .& (df_recConst.need .== j), :quantity][1] for (j,k) in ir)

#quantity of calories per unit of macros
qtycal = Dict("protein" => 4, "carb" => 5, "fat" => 6)

#minimum contribution from each macro to calories
ratio = Dict("protein" => 0.10, "carb" => 0.45, "fat" => 0.20)

#maximum contribution from each macro to calories
ratio2 = Dict("protein" => 13, "carb" => 13, "fat" => 13)

#minimum percentage of calories in each meal
perc = [0.10,0.30,0.30]

#maximum percentage of calories in each meal
perc2 = [0.25,0.70,0.50]

#maximum percentage of meals for which the same recipe can be chosen
samerecperc = 0.6

println(nutrec)

Dict{Tuple{String15, String31}, Int64}(("calories", "beef_with_rice") => 73, ("carb", "pancakes") => 9, ("carb", "oatmeal") => 10, ("calories", "pancakes") => 23, ("VitA", "chicken_with_potatoes") => 1, ("VitA", "pancakes") => 1, ("VitA", "oatmeal") => 1, ("protein", "chicken_with_potatoes") => 30, ("VitB", "beef_with_rice") => 3, ("protein", "pancakes") => 8, ("VitA", "beef_with_rice") => 2, ("fat", "pancakes") => 7, ("fat", "oatmeal") => 10, ("VitB", "chicken_with_potatoes") => 1, ("VitB", "pancakes") => 1, ("fat", "chicken_with_potatoes") => 10, ("VitB", "oatmeal") => 1, ("fat", "beef_with_rice") => 15, ("carb", "chicken_with_potatoes") => 20, ("calories", "chicken_with_potatoes") => 70, ("calories", "oatmeal") => 20, ("carb", "beef_with_rice") => 35, ("protein", "beef_with_rice") => 25, ("protein", "oatmeal") => 10)


In [100]:
#DEFINE AND SHOW MODEL

#--model
mdl = Model(with_optimizer(Gurobi.Optimizer))

#--decision variables
@variables mdl begin
    X[rmd], Bin
    Q[rmdp]>=0
    Y[I]>=0 #need to understand if it is possible to define some of these as integer and others as continuous
end

#change variable Y to be integer for certain ingredients
for s in I
    if df_price[(df_price.ingredient .== s), :unit][1] == "each" || df_price[(df_price.ingredient .== s), :unit][1] == "ct"
        set_integer(Y[s])
    end
end

#--objective function
@objective(mdl, Min, sum(Y[j]*price[j] for j in I))

#--constraints
@constraints mdl begin
    constraint_1[i in M,j in D],
    sum(X[(l,v,s)] for (l,v,s) in rmd if v==i && s==j) == 1
    
    constraint_2[i in D,j in N,l in P],
    sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l) >= need[(j,l)] 

    constraint_3[(i,j,l) in rmd],
    sum(Q[(i,j,l,k)] for k in P) <= 1000*X[(i,j,l)]
    
    constraint_4[s in I],
    sum(Q[(i,j,l,k)]*recing[(s,i)] for (i,j,l,k) in rmdp if s in mydict[i]) <= Y[s]
    
    constraint_5[i in D,j in uppnut,l in P],
    sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l) <= max[(j,l)]
    
    constraint_6[k in M,i in D,j in uppnutmeal,l in P], #this is supposed to be calories instead of protein
    sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l && v==k) >= perc[k]*need[(j,l)]
    
    constraint_7[k in M,i in D,j in uppnutmeal,l in P], #this is supposed to be calories instead of protein
    sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==i && a==l && v==k) <= perc2[k]*need[(j,l)]
    
    constraint_8[i in mydict2[2]],
    sum(X[(h,v,s)] for (h,v,s) in rmd if h==i) - samerecperc*(d*(m-1)) <= 0 #(v in [2,3] &&)
    
    constraint_9[i in mydict2[2], j in D],
    sum(X[(h,v,s)] for (h,v,s) in rmd if h==i && s==j) <= 1
    
    constraint_10[i in ["protein","carb","fat"],j in ["calories"], k in D, l in P],
    sum(Q[(h,v,s,a)]*nutrec[(i,h)]*qtycal[i] for (h,v,s,a) in rmdp if s==k && a==l) >= ratio[i]*sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==k && a==l)
    #ratio[i]*sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==k && a==l) >= 0
    
    constraint_11[i in ["protein","carb","fat"],j in ["calories"], k in D, l in P],
    sum(Q[(h,v,s,a)]*nutrec[(i,h)]*qtycal[i] for (h,v,s,a) in rmdp if s==k && a==l) <= ratio2[i]*sum(Q[(h,v,s,a)]*nutrec[(j,h)] for (h,v,s,a) in rmdp if s==k && a==l)
end

Set parameter Username
Academic license - for non-commercial use only - expires 2023-03-27


(2-dimensional DenseAxisArray{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape},2,...} with index sets:
    Dimension 1, [1, 2, 3]
    Dimension 2, [1, 2]
And data, a 3×2 Matrix{ConstraintRef{Model, MathOptInterface.ConstraintIndex{MathOptInterface.ScalarAffineFunction{Float64}, MathOptInterface.EqualTo{Float64}}, ScalarShape}}:
 constraint_1[1,1] : X[("oatmeal", 1, 1)] + X[("pancakes", 1, 1)] = 1.0                      …  constraint_1[1,2] : X[("oatmeal", 1, 2)] + X[("pancakes", 1, 2)] = 1.0
 constraint_1[2,1] : X[("chicken_with_potatoes", 2, 1)] + X[("beef_with_rice", 2, 1)] = 1.0     constraint_1[2,2] : X[("chicken_with_potatoes", 2, 2)] + X[("beef_with_rice", 2, 2)] = 1.0
 constraint_1[3,1] : X[("chicken_with_potatoes", 3, 1)] + X[("beef_with_rice", 3, 1)] = 1.0     constraint_1[3,2] : X[("chicken_with_potatoes", 3, 2)] + X[("beef_with_rice", 3, 2)] = 1.0, 3-dimensional DenseAxisArr

In [101]:
#--solve the model
optimize!(mdl)

choosen_recipes = value.(X)
quantities = value.(Q)
needed_ing = value.(Y)

result = objective_value(mdl)

println("Choosen recipes: ", choosen_recipes)
println("Quantities of recipes: ", quantities)
println("Needed_ingredients: ", needed_ing)
println("Cost: ", result)

Gurobi Optimizer version 9.5.1 build v9.5.1rc2 (mac64[arm])
Thread count: 10 physical cores, 10 logical processors, using up to 10 threads
Optimize a model with 114 rows, 42 columns and 518 nonzeros
Model fingerprint: 0x721cf7b0
Variable types: 28 continuous, 14 integer (12 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+03]
  Objective range  [2e+00, 4e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+00, 1e+02]
Found heuristic solution: objective 300.4954991
Presolve removed 70 rows and 12 columns
Presolve time: 0.00s
Presolved: 44 rows, 30 columns, 134 nonzeros
Found heuristic solution: objective 297.4954990
Variable types: 24 continuous, 6 integer (4 binary)
Found heuristic solution: objective 294.4954990

Root relaxation: objective 2.237184e+02, 23 iterations, 0.00 seconds (0.00 work units)

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0  2